# Python a C

V pythonu je více možnjostí, jak volat C funkce z pythoního kódu (modul `ctypes`, SWIG, python/C API).

**Proč to dělat?** Rychlost (když pypy nebo cython nestačí), potřeba použít existující obskurní či osvědčenou knihovnu (\*.SO/\*.DLL), *protože proto*! ;-)

## Zdroje k studiu:

*   https://docs.python.org/3.8/extending/extending.html
*   https://www.journaldev.com/31907/calling-c-functions-from-python
*   https://book.pythontips.com/en/latest/python_c_extension.html (odtud pochází příklad pro Python/C API)

---


# ctypes

In [80]:
%%writefile mylib.c
#include <stdio.h>
#include <math.h>
int rnd();
int add(int, int);
float fadd(float, float);
float fsqrt(float);
double PI = 3.14;

int rnd() {
	return 9;  //# https://dilbert.com/strip/2001-10-25
}

int add(int a, int b) {
	return a + b;
}

float fl_add(float a, float b) {
	return a + b;
}

float fsqrt(float a) {
	return sqrt(a);
}

Overwriting mylib.c


In [81]:
!rm mylib.so 2>&1 > /dev/null
# -fPIC ... https://gcc.gnu.org/onlinedocs/gcc/Code-Gen-Options.html#Code-Gen-Options
!gcc -fPIC -shared -Wl,-soname,mylib -o mylib.so mylib.c
!ls -la

total 304
drwxr-xr-x 1 root root   4096 May 16 13:25 .
drwxr-xr-x 1 root root   4096 May 16 12:02 ..
-rw-r--r-- 1 root root   1567 May 16 13:03 adder.c
drwxr-xr-x 2 root root   4096 May 16 13:03 addList.egg-info
drwxr-xr-x 5 root root   4096 May 16 12:50 build
drwxr-xr-x 4 root root   4096 May 12 13:30 .config
drwxr-xr-x 2 root root   4096 May 16 12:50 dist
-rw-r--r-- 1 root root    325 May 16 12:43 mylib2.i
-rw-r--r-- 1 root root   2227 May 16 12:43 mylib2.py
-rwxr-xr-x 1 root root  59384 May 16 12:43 _mylib2.so
-rw-r--r-- 1 root root 112091 May 16 12:43 mylib2_wrap.c
-rw-r--r-- 1 root root  54408 May 16 12:43 mylib2_wrap.o
-rw-r--r-- 1 root root    335 May 16 13:24 mylib.c
-rw-r--r-- 1 root root   1912 May 16 12:43 mylib.o
-rwxr-xr-x 1 root root  15984 May 16 13:25 mylib.so
drwxr-xr-x 2 root root   4096 May 16 12:43 __pycache__
drwxr-xr-x 1 root root   4096 May 12 13:31 sample_data
-rw-r--r-- 1 root root    130 May 16 12:57 setup.py
-rw-r--r-- 1 root root    328 May 16 13:03 test_add

In [2]:
import ctypes
from ctypes import *

# reloading magic - https://stackoverflow.com/a/62021495
def ctypes_close_library(lib):
    dlclose_func = ctypes.CDLL(None).dlclose
    dlclose_func.argtypes = [ctypes.c_void_p]
    dlclose_func.restype = ctypes.c_int
    dlclose_func(lib._handle)

ml = CDLL("./mylib.so")
# ml.fsqrt.argtypes = (c_float,) # spatny vysledek pri volani s intem
ml.fsqrt.restype = c_float
ml.fl_add.argtypes = (c_float, c_float)
ml.fl_add.restype = c_float

print(type(ml))
print(f'random number by C code = {ml.rnd(10)}')
print(f'ints A + B by C code = {ml.add(22, 20)}')
print(f'floats A + B by C code = {ml.fl_add(22, 20.0)}')  # nekompatibilni typy! -> nutno uvest pomoci: ml.fadd.argtypes = (c_float, c_float)
print(f'floats A + B by C code = {ml.fl_add(c_float(22.0), c_float(20))}')  
print(f'float sqrt = {ml.fsqrt(c_float(25.0))}')  
print(f'int sqrt = {ml.fsqrt(25)}')  

ctypes_close_library(ml)

<class 'ctypes.CDLL'>
random number by C code = 9
ints A + B by C code = 42
floats A + B by C code = 42.0
floats A + B by C code = 42.0
float sqrt = 5.0
int sqrt = 0.0


# SWIG

[SWIG](https://www.swig.org) = Simplified Wrapper and Interface Generator, se nejspíše vyplatí použít, pokud chcete svůj C/C++ kód zpřístupňovat ve více jazycích. Jednou napíšete interface a pak už generujete wrappery.

In [3]:
!apt-get install swig
#!apt python3.7-dev libpython3.7-dev
!python --version
#!apt search python3.10-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
swig is already the newest version (4.0.1-5build1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Python 3.10.11


In [4]:
%%writefile mylib2.i

/* mylib2.i */
 %module mylib2
 %{
 /* Put header files here or function declarations like below */
 extern double PI;
 extern int rnd();
 extern int add(int a, int b);
 extern float fl_add(float a, float b);
 %}
 
 extern double PI;
 extern int rnd();
 extern int add(int a, int b);
 extern float fl_add(float a, float b);

Overwriting mylib2.i


In [5]:
#!rm ./mylib*
!ls -la
!swig -python mylib2.i
!ls -la

total 304
drwxr-xr-x 1 root root   4096 May 16 13:25 .
drwxr-xr-x 1 root root   4096 May 16 12:02 ..
-rw-r--r-- 1 root root   1567 May 16 13:03 adder.c
drwxr-xr-x 2 root root   4096 May 16 13:03 addList.egg-info
drwxr-xr-x 5 root root   4096 May 16 12:50 build
drwxr-xr-x 4 root root   4096 May 12 13:30 .config
drwxr-xr-x 2 root root   4096 May 16 12:50 dist
-rw-r--r-- 1 root root    325 May 16 13:35 mylib2.i
-rw-r--r-- 1 root root   2227 May 16 12:43 mylib2.py
-rwxr-xr-x 1 root root  59384 May 16 12:43 _mylib2.so
-rw-r--r-- 1 root root 112091 May 16 12:43 mylib2_wrap.c
-rw-r--r-- 1 root root  54408 May 16 12:43 mylib2_wrap.o
-rw-r--r-- 1 root root    335 May 16 13:24 mylib.c
-rw-r--r-- 1 root root   1912 May 16 12:43 mylib.o
-rwxr-xr-x 1 root root  15984 May 16 13:25 mylib.so
drwxr-xr-x 2 root root   4096 May 16 12:43 __pycache__
drwxr-xr-x 1 root root   4096 May 12 13:31 sample_data
-rw-r--r-- 1 root root    130 May 16 12:57 setup.py
-rw-r--r-- 1 root root    328 May 16 13:03 test_add

Přibylo **mylib2.py**, podíváme se na něj:

In [6]:
!cat mylib2.py

# This file was automatically generated by SWIG (http://www.swig.org).
# Version 4.0.1
#
# Do not make changes to this file unless you know what you are doing--modify
# the SWIG interface file instead.

from sys import version_info as _swig_python_version_info
if _swig_python_version_info < (2, 7, 0):
    raise RuntimeError("Python 2.7 or later required")

# Import the low-level C/C++ module
if __package__ or "." in __name__:
    from . import _mylib2
else:
    import _mylib2

try:
    import builtins as __builtin__
except ImportError:
    import __builtin__

def _swig_repr(self):
    try:
        strthis = "proxy of " + self.this.__repr__()
    except __builtin__.Exception:
        strthis = ""
    return "<%s.%s; %s >" % (self.__class__.__module__, self.__class__.__name__, strthis,)


def _swig_setattr_nondynamic_instance_variable(set):
    def set_instance_attr(self, name, value):
        if name == "thisown":
            self.this.own(value)
        elif name == "this":
           

Vyrobíme **mylib2.so**:

In [7]:
#!find / -name Python.h
!gcc -fPIC -c mylib.c mylib2_wrap.c -I/usr/include/python3.10/
!ld -shared mylib.o mylib2_wrap.o -o _mylib2.so
#!ls -la
#!cat mylib2_wrap.c | awk 'NR == 150, NR == 160 { print NR, $0 }' 

A hurá to použít

In [8]:
import mylib2 as ml2
print(f'random number by C code = {ml2.rnd()}')
print(f'ints A + B by C code = {ml2.add(22, 20)}')
print(f'floats A + B by C code = {ml2.fl_add(22.0, 20.0)}')

random number by C code = 9
ints A + B by C code = 42
floats A + B by C code = 42.0


# Python/C API

Poslední možnost je asi nejkomplikovanější, ale na druhou stranu nám umožňuje jako jediná z uvedených modifikovat pythonové objekty v C kódu.

Základem je `Python.h` header file, který poskytuje funkce (+/- ekvivalenty k funkcím z `builtins`) k manipulaci se strukturou `PyObject`. Ta reprezentuje libovolný pythonový objekt. Příkladem budiž `PyList_Size()`, která počítá délku `PyListType` (tedy listu).

Napíšeme si rozšíření pythonu v C, které sečte prvky v listu (předpokládáme numerické hodnoty). Začneme s výsledným použitím:

In [9]:
%%writefile test_adder.py
import addList
l = [1, 2, 3, 4, 5]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [1, 2, 3, 4, 5, 'bug', 6, 7]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = ['a', 'b']
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [42, 42**12]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")

Overwriting test_adder.py


Teď je na řadě samotná implementace v C:

In [10]:
%%writefile adder.c
#define PY_SSIZE_T_CLEAN
#include <Python.h>

// hlavni funkce volana z pythonu, konvece pojmenoveni je:
// {module-name}_{function-name}
static PyObject* addList_add(PyObject* self, PyObject* args){
  return Py_BuildValue("i", 42); // prozatim vratime python objekt integer...
}

static PyMethodDef module_functions[] = {
    {"add", (PyCFunction)addList_add, METH_VARARGS, PyDoc_STR("sums the list")},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

static struct PyModuleDef cAddList =
{
    PyModuleDef_HEAD_INIT,
    "addList",   /* name of module */
    "",          /* module documentation, may be NULL */
    -1,          /* size of per-interpreter state of the module, or -1 if the module keeps state in global variables. */
    module_functions
};

PyMODINIT_FUNC PyInit_addList(void)
{
    return PyModule_Create(&cAddList);
}


Overwriting adder.c


Jdeme to zkompilovat:

In [4]:
%%writefile setup.py
from distutils.core import setup, Extension
setup(name='addList', version='1.0', ext_modules=[Extension('addList', ['adder.c'])])

Writing setup.py


In [12]:
import sys
import subprocess
!python3 setup.py install
r = subprocess.run([sys.executable, "test_adder.py"], capture_output=True)
print(f"Output:\n{r.stdout.decode('UTF-8')}")

/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        S

A pojďme doplnit i samotný výkonný kód, který nám posčítá prvky listu:


*   [PyArg_ParseTuple() string arguments](https://docs.python.org/3/c-api/arg.html)

In [6]:
%%writefile adder.c
#define PY_SSIZE_T_CLEAN
#include <Python.h>

// hlavni funkce volana z pythonu, konvece pojmenoveni je:
// {module-name}_{function-name}
static PyObject* addList_add(PyObject* self, PyObject* args){
  PyObject * listObj;

  //The input arguments come as a tuple, we parse the args to get the various variables
  //In this case it's only one list variable, which will now be referenced by listObj
  if (! PyArg_ParseTuple(args, "O", &listObj))
    return NULL;

  Py_ssize_t i, n;
  n = PyList_Size(listObj);
  if (n < 0)
    Py_RETURN_NONE; // listObj is not a list 

  long sum = 0, elem;
  for(i = 0; i < n; i++){
    PyObject* temp = PyList_GetItem(listObj, i);
    if (!PyLong_Check(temp)) continue; // lets skip non-integers
    elem = PyLong_AsLong(temp);
    // printf("e=%dl", elem);
    if (elem==-1 && PyErr_Occurred())
      Py_RETURN_NONE; // number not fit into C long
    sum += elem;
  }

  // value returned back to python code - another python object
  return Py_BuildValue("i", sum);
}

static PyMethodDef module_functions[] = {
    {"add", (PyCFunction)addList_add, METH_VARARGS, PyDoc_STR("sums the list")},
    {NULL, NULL, 0, NULL}        /* Sentinel */
};

static struct PyModuleDef cAddList =
{
    PyModuleDef_HEAD_INIT,
    "addList",   /* name of module */
    "",          /* module documentation, may be NULL */
    -1,          /* size of per-interpreter state of the module, or -1 if the module keeps state in global variables. */
    module_functions
};

PyMODINIT_FUNC PyInit_addList(void)
{
    return PyModule_Create(&cAddList);
}

Overwriting adder.c


In [2]:
%%writefile test_adder.py
import addList
l = [1, 2, 3, 4, 5]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [1, 2, 3, 4, 5, 'bug', 6, 7]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = ['a', 'b']
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")
l = [42, 42**12]
print(f"Sum of List - {str(l)} = {str(addList.add(l))}")


Writing test_adder.py


In [7]:
import sys
import subprocess
!python3 setup.py install
!date
!ls -la
r = subprocess.run([sys.executable, "test_adder.py"], capture_output=True)
print(r)
print(f"Output:\n{r.stdout.decode('UTF-8')}")
print(f"Error:\n{r.stderr.decode('UTF-8')}" if len(r.stderr) > 0 else '') 

/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` directly.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        See https://blog.ganssle.io/articles/2021/10/setup-py-deprecated.html for details.
        ********************************************************************************

!!
  self.initialize_options()
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: EasyInstallDeprecationWarning: easy_install command is deprecated.
!!

        ********************************************************************************
        Please avoid running ``setup.py`` and ``easy_install``.
        Instead, use pypa/build, pypa/installer, pypa/build or
        other standards-based tools.

        S